In [1]:
# main.ipynb

# Импортируем функцию для загрузки данных
from Modules.data_loader import load_data

# Выводим сообщение перед загрузкой данных
print("Начинаем загрузку данных...")

# Загрузим данные и проверим время выполнения
import time
start_time = time.time()

X = load_data()

end_time = time.time()
print(f"Данные загружены успешно за {end_time - start_time:.2f} секунд.")

# Проверим, что данные загружены корректно
if X is not None:
    print("Данные загружены корректно:")
    print(X.head())
else:
    print("Ошибка: данные не загружены.")


Начинаем загрузку данных...
Данные загружены из репозитория и сохранены в файл.
Данные загружены успешно за 88.46 секунд.
Данные загружены корректно:
                           Description  Quantity     InvoiceDate  UnitPrice  \
0   WHITE HANGING HEART T-LIGHT HOLDER         6  12/1/2010 8:26       2.55   
1                  WHITE METAL LANTERN         6  12/1/2010 8:26       3.39   
2       CREAM CUPID HEARTS COAT HANGER         8  12/1/2010 8:26       2.75   
3  KNITTED UNION FLAG HOT WATER BOTTLE         6  12/1/2010 8:26       3.39   
4       RED WOOLLY HOTTIE WHITE HEART.         6  12/1/2010 8:26       3.39   

   CustomerID         Country  
0     17850.0  United Kingdom  
1     17850.0  United Kingdom  
2     17850.0  United Kingdom  
3     17850.0  United Kingdom  
4     17850.0  United Kingdom  


In [2]:
# Проверка информации о данных
print(X.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Description  540455 non-null  object 
 1   Quantity     541909 non-null  int64  
 2   InvoiceDate  541909 non-null  object 
 3   UnitPrice    541909 non-null  float64
 4   CustomerID   406829 non-null  float64
 5   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(3)
memory usage: 24.8+ MB
None


In [3]:
# Вывод первых строк данных
print(X.head())


                           Description  Quantity     InvoiceDate  UnitPrice  \
0   WHITE HANGING HEART T-LIGHT HOLDER         6  12/1/2010 8:26       2.55   
1                  WHITE METAL LANTERN         6  12/1/2010 8:26       3.39   
2       CREAM CUPID HEARTS COAT HANGER         8  12/1/2010 8:26       2.75   
3  KNITTED UNION FLAG HOT WATER BOTTLE         6  12/1/2010 8:26       3.39   
4       RED WOOLLY HOTTIE WHITE HEART.         6  12/1/2010 8:26       3.39   

   CustomerID         Country  
0     17850.0  United Kingdom  
1     17850.0  United Kingdom  
2     17850.0  United Kingdom  
3     17850.0  United Kingdom  
4     17850.0  United Kingdom  


In [4]:
# Описание данных для числовых признаков
print(X.describe())


            Quantity      UnitPrice     CustomerID
count  541909.000000  541909.000000  406829.000000
mean        9.552250       4.611114   15287.690570
std       218.081158      96.759853    1713.600303
min    -80995.000000  -11062.060000   12346.000000
25%         1.000000       1.250000   13953.000000
50%         3.000000       2.080000   15152.000000
75%        10.000000       4.130000   16791.000000
max     80995.000000   38970.000000   18287.000000


In [5]:
# Проверка наличия пропущенных значений
print(X.isnull().sum())


Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64


In [6]:
# Отбор строк без CustomerID
no_customer_data = X[X['CustomerID'].isnull()]

# Вывод первых строк данных без CustomerID
print(no_customer_data.head())


                          Description  Quantity      InvoiceDate  UnitPrice  \
622                               NaN        56  12/1/2010 11:52       0.00   
1443  DECORATIVE ROSE BATHROOM BOTTLE         1  12/1/2010 14:32       2.51   
1444  DECORATIVE CATS BATHROOM BOTTLE         2  12/1/2010 14:32       2.51   
1445               POLKADOT RAIN HAT          4  12/1/2010 14:32       0.85   
1446            RAIN PONCHO RETROSPOT         2  12/1/2010 14:32       1.66   

      CustomerID         Country  
622          NaN  United Kingdom  
1443         NaN  United Kingdom  
1444         NaN  United Kingdom  
1445         NaN  United Kingdom  
1446         NaN  United Kingdom  


In [7]:
# Проверка наличия всех полей, которые были в датасете из ZIP-архива
required_columns = ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']
missing_columns = [col for col in required_columns if col not in X.columns]
print("Отсутствующие поля:", missing_columns)

Отсутствующие поля: ['InvoiceNo', 'StockCode']


In [1]:
# Импорт необходимых библиотек
import pandas as pd

# Загрузка данных из репозитория UCI
from ucimlrepo import fetch_ucirepo

# Загрузка датасета
online_retail = fetch_ucirepo(id=352)

# Проверка содержимого targets
y = online_retail.data.targets
print("Содержимое targets:")
print(y)


Содержимое targets:
None
